In [7]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import scipy.io as sio
import os
from scipy import stats
import shutil
import time

ModuleNotFoundError: No module named 'cgi'

In [5]:
pip install mne==0.24.1 pandas numpy scipy matplotlib seaborn networkx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Realice un preprocesado a los datos, ya que venían en crudo

In [1]:
raw_folder = r"C:\Users\HP\Documents\GitHub\Neurociencias-2026-1\S06_proyecto\datasetp\DESTINATION\EEG"
procesado = r"C:\Users\HP\Documents\GitHub\Neurociencias-2026-1\S06_proyecto\datasetp\DESTINATION\eeg_preprocesado"

# Canales potencialmente malos
bad_channels_list = ['F8', 'POz', 'AF4', 'AF8', 'F6', 'FC3']

# Crear carpeta de preprocesados si no existe
os.makedirs(procesado, exist_ok=True)

# Lista para metadata
metadata_list = []

# Recorrer todos los archivos EDF
for filename in os.listdir(raw_folder):
    if filename.endswith(".edf"):
        filepath = os.path.join(raw_folder, filename)
        
        # Extraer información de nombre: XX_YY_Z.edf
        base = filename.replace(".edf", "")
        parts = base.split("_")
        if len(parts) != 3:
            print(f"Archivo {filename} no sigue el patrón XX_YY_Z, se omite")
            continue

        subj, task, attempt = parts
        # Crear carpeta de sujeto y tarea
        subj_folder = os.path.join(procesado, f"Sujeto_{subj}")
        task_folder = os.path.join(subj_folder, f"Tarea_{task}")
        os.makedirs(task_folder, exist_ok=True)

        # Leer EDF
        try:
            raw = mne.io.read_raw_edf(filepath, preload=True)
        except Exception as e:
            print(f"No se pudo leer {filename}: {e}")
            continue

        # Marcar canales malos si existen
        raw.info['bads'] = [ch for ch in bad_channels_list if ch in raw.ch_names]

        # Guardar como .fif
        save_path = os.path.join(task_folder, f"{filename.replace('.edf','.fif')}")
        raw.save(save_path, overwrite=True)

        # Metadata
        duration_s = raw.n_times / raw.info['sfreq']
        n_channels = raw.info['nchan']
        bads = ",".join(raw.info['bads']) if raw.info['bads'] else ""

        metadata_list.append({
            "participantID": subj,
            "task": task,
            "attempt": attempt,
            "duration_s": duration_s,
            "n_channels": n_channels,
            "bad_channels": bads,
            "file_path": save_path
        })

        print(f"Procesado: {filename}")

# Guardar CSV de metadata
df_metadata = pd.DataFrame(metadata_list)
metadata_csv_path = os.path.join(procesado, "metadata.csv")
df_metadata.to_csv(metadata_csv_path, index=False)
print(f"Metadata guardada en: {metadata_csv_path}")

NameError: name 'os' is not defined